В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [1]:
# Імпорт загальних бібліотек

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Зчитування даних

raw_df = pd.read_csv('./bank-customer-churn-prediction-dlu-course-c-2/train.csv')

# Задання початкових умов

drop_cols = ['id', 'CustomerId', 'Surname']
scaler_numeric = False

In [3]:
# Імпорт функції preprocess_data з process_bank_churn.py

%load_ext autoreload
%autoreload 2

from process_bank_churn import preprocess_data

In [4]:
data, input_cols, scaler, encoder = preprocess_data(raw_df, drop_cols, scaler_numeric)

In [5]:
X_train = data['train_X']
train_targets = data['train_y']
X_val = data['val_X']
val_targets = data['val_y']

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [6]:
# Навчання класифікатора kNN

knn = KNeighborsClassifier()
knn.fit(X_train, train_targets)

KNeighborsClassifier()

In [7]:
# Функція для підрахунку метрики AUROC

def pred_auroc(model, inputs, targets, name = ''):

    # Predict probabilities
    pred_proba = model.predict_proba(inputs)[:, 1]
    
    # Compute ROC curve
    fpr, tpr, threshold = roc_curve(targets, pred_proba, pos_label = 1)
    
    # Compute AUROC
    roc_auc = auc(fpr, tpr)
    print(f"Area under ROC score on {name} dataset: {roc_auc:.2%}")
  
    return roc_auc

In [8]:
# Розрахунок точності на тренувальному і валідаційному наборах за допомогою AUROC

roc_auc_train = pred_auroc(knn, X_train, train_targets, 'Training')
roc_auc_val = pred_auroc(knn, X_val, val_targets, 'Validation')

Area under ROC score on Training dataset: 82.28%
Area under ROC score on Validation dataset: 57.83%


**Висновок**:\
Враховуючи, що `AUROC` на валідаційних даних (57,83%) значно нижчий, ніж на тренувальних (82,28%), можна зробити висновок, що модель перетренована.  

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [9]:
# Пошук оптимального значення параметра n_neighbors для класифікатора kNN 

params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [10]:
# Найкраща модель, найкраще значення параметра

knn_best = knn_gs.best_estimator_

print(knn_gs.best_params_)

{'n_neighbors': 24}


In [11]:
# Оцінка якості передбачень knn_best на тренувальній і валідаційній вибірці з допомогою AUROC

roc_auc_train = pred_auroc(knn_best, X_train, train_targets, 'Training')
roc_auc_val = pred_auroc(knn_best, X_val, val_targets, 'Validation')

Area under ROC score on Training dataset: 68.85%
Area under ROC score on Validation dataset: 60.86%


**Висновок**:\
Модель незначно покращилась в порівнянні з попердньою (AUROC на валідаційному наборі 60,86% проти 57,83%).\
Але стала значно гіршою за результат, який показало дерево прийняття рішень в попередньому ДЗ (там AUROC на валідаційних даних перевищував 92%).

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [12]:
%%time
dt = DecisionTreeClassifier(random_state = 42)
params_dt = {'max_depth': np.arange(1, 20, 2), 'max_leaf_nodes': np.arange(2, 10, 1)}
dt_gs = GridSearchCV(dt, params_dt, cv = 3, scoring='roc_auc')
dt_gs.fit(X_train, train_targets)

Wall time: 3.43 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'max_leaf_nodes': array([2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='roc_auc')

In [13]:
# Найкраща модель, найкращі значення параметрів

dt_best = dt_gs.best_estimator_

print(dt_gs.best_params_)

{'max_depth': 5, 'max_leaf_nodes': 9}


In [14]:
# Оцінка якості передбачень dt_best на тренувальній і валідаційній вибірці з допомогою AUROC

roc_auc_train = pred_auroc(dt_best, X_train, train_targets, 'Training')
roc_auc_val = pred_auroc(dt_best, X_val, val_targets, 'Validation')

Area under ROC score on Training dataset: 89.97%
Area under ROC score on Validation dataset: 90.01%


**Висновок**:\
В моделі, де параметри підбирались вручну (попереднє ДЗ), `AUROC` на тренувальних даних становив 92,89%, на валідаційних – 92,34%. Виходячи з цього, остання модель (`AUROC` на тренувальних даних 89,97% і на валідаційних 90,01%) не є кращою за ту, що була знайдена вручну.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [15]:
%%time
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}
search = RandomizedSearchCV(dt, params_dt, n_iter = 40, cv = 3, scoring = 'roc_auc', verbose = 0, refit = True)
search.fit(X_train, train_targets)

Wall time: 1.08 s


RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=40,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   scoring='roc_auc')

In [16]:
# Найкраща модель, найкращі значення параметрів

dt_random_search_best = search.best_estimator_

print(search.best_params_)

{'splitter': 'random', 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_leaf_nodes': 16, 'max_features': None, 'max_depth': 11, 'criterion': 'entropy'}


In [17]:
# Оцінка якості передбачень dt_random_search_best на тренувальній і валідаційній вибірці з допомогою AUROC

roc_auc_train = pred_auroc(dt_random_search_best, X_train, train_targets, 'Training')
roc_auc_val = pred_auroc(dt_random_search_best, X_val, val_targets, 'Validation')

Area under ROC score on Training dataset: 90.74%
Area under ROC score on Validation dataset: 90.83%


**Висновок**:
1) Модель якісна, добре генералізує. Порівнюючи метрику `AUROC` на тренувальних і валідаційних даних з відповідними значеннями, які були знайдені з GridSearch вище, можна стверджувати що остання модель трохи краща. Проте вона все ще програє моделі з попереднього ДЗ, де параметри дерева рішень підбирались вручну (там AUROC на тренувальних даних становив 92,89%, на валідаційних – 92,34%).
2) Порівнюючи параметри `dt_best` і `dt_random_search_best`, бачимо відмінність у `max_depth` (5 у першому випадку, 11 – у другому) і `max_leaf_nodes` (9 у першому випадку, 16 – у другому).

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [18]:
# Зчитування тестових даних

new_raw_df = pd.read_csv('./bank-customer-churn-prediction-dlu-course-c-2/test.csv')

In [19]:
# Імпорт функції preprocess_new_data з process_bank_churn.py для обробки тестового набору

from process_bank_churn import preprocess_new_data

In [20]:
# Обробка тестового набору

new_data  = preprocess_new_data(new_raw_df, drop_cols, scaler_numeric, scaler, encoder)

In [21]:
X_test = new_data['test_X']

In [22]:
# Знаходження передбачень на тестових даних, формування файлу submission

new_raw_df['Exited'] = dt_random_search_best.predict_proba(X_test)[:, 1]

submission_df = pd.read_csv('./bank-customer-churn-prediction-dlu-course-c-2/sample_submission.csv')
submission_df.Exited = new_raw_df.Exited
submission_df.to_csv('submission.csv', index = False)

Оскільки в останній серії експериментів метрика`AUROC` програє тій, що була знайдена в попередніх ДЗ, `submission` на Kaggle зараз не робила. 